# Testing

## Why testing

- Proove that your code works.
- Make modification easy.
- Document expected behaviour.
- Easy to do.
- Methodology.

In [ ]:
from datetime import datetime
def ymd(date_string):
    """
    Example:
    
    >>> ymd('2016 01 30')
    datetime.datetime(2016, 1, 30, 0, 0)
    """
    format_string = '%Y %m %d'
    return datetime.strptime(date_string, format_string)

In [ ]:
ymd('2016 01 30')

In [ ]:
import unittest

In [ ]:
class YMDTest(unittest.TestCase):
    def test_with_known_values(self):
        ymd_result = ymd('2016 01 30')
        known = datetime(2016, 1, 30, 0, 0)
        self.assertEqual(ymd_result, known)

# Running

    python -m unittest YMDTest

In [ ]:
result = unittest.TestResult()
test = YMDTest('test_with_known_values')
test.run(result)
result

In [ ]:
def run_test(cls, method_name):
    result = unittest.TestResult()
    test = cls(method_name)
    test.run(result)
    return result

In [ ]:
run_test(YMDTest, 'test_with_known_values')

In [ ]:
class YMDError(YMDTest):
    def test_throws_on_invalid_string(self):
        with self.assertRaises(ValueError):
            ymd('')
        with self.assertRaises(ValueError):
            # day out of range
            ymd_result = ymd('2016 02 30')

In [ ]:
run_test(YMDError, 'test_throws_on_invalid_string')

# What you can test

- Pipelines
- Data prep & transformation
- Data ingestion
- ...

# What you can't test

- Prediction accuracy (within limits)
    - Disaster prevention is possible
- 'Statistical stuff'

In [ ]:
class MyTests(unittest.TestCase):
    def test_stuff(self):
        self.assertAlmostEqual(0.00000000001, 0)
        self.assertEqual(1, 1)
        self.assertEqual([1,2], [1,2])
        self.assertEqual({1: 1}, {1: 1})
        
        self.assertTrue(1 == 1) # avoid
        self.assertTrue(1 in range(2)) # avoid
        self.assertIn(1, range(2)) # better
        
        self.assertTrue(1 < 2) # avoid
        self.assertLess(1, 2) # better

In [ ]:
run_test(MyTests, 'test_stuff')

In [ ]:
for i in dir(unittest.TestCase):
    if i.startswith('assert'):
        print i

In [ ]:
import doctest

In [ ]:
doctest.run_docstring_examples(ymd, globals(), True)

# Twitter

In [ ]:
import requests
from requests_oauthlib import OAuth1

In [ ]:
from donthackme import CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET

In [ ]:
# api url
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'

In [ ]:
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [ ]:
verify_response = requests.get(url, auth=auth)

In [ ]:
verify_response

In [ ]:
verify_response.headers

In [ ]:
verify_response.json().keys()

In [ ]:
verify_response.json()['name']

In [ ]:
verify_response.json()['screen_name']

# Search

In [ ]:
search_url = 'https://api.twitter.com/1.1/search/tweets.json'

In [ ]:
params = {'q': 'data science',
          'result_type': 'recent'} # popular also possible

In [ ]:
search_response = requests.get(search_url, params=params, auth=auth)

In [ ]:
search_response

In [ ]:
search_response.json()

In [ ]:
re_json = search_response.json()

In [ ]:
first_status = re_json.get('statuses', [{}])[0]
print first_status['text']

In [ ]:
sorted(first_status.keys())

In [ ]:
first_status['user'].keys()

In [ ]:
# okay: re_json['statuses']
re_json.get('statuses', [{}])[0] #advanced

## Streaming API

In [ ]:
from itertools import islice

In [ ]:
r = requests.post('https://stream.twitter.com/1.1/statuses/filter.json',
                 params = {'track': '#data'},
                 auth=auth,
                 stream=True) # important

In [ ]:
tweets = r.iter_lines()

In [ ]:
import json

In [ ]:
for tweet in islice(tweets, 20):
    if tweet != None:
        print json.loads(tweet)['text'][:20]
    else:
        print 'Timeout.'

In [ ]:
r.close()

In [ ]:
# student question: What about the u in u'string'?
print u'Hi, Håvard!'

In [ ]:
u'Hi, Håvard!' # good

In [ ]:
'Hi, Håvard!' # less good!

In [ ]:
type(u'')

In [ ]:
type('')

# Storing things

## Files (e.g. .csv)

- Don't scale (could use HDFS)
- What about JSON?

## Databases

- SQL
- NoSQL

# MongoDB

- Windows: Download 'Community edition'
- MAC
    brew update
    brew install mongodb
    brew services start mongobd
- Linux
    sudo apt install mongodb
    
Plus:

    pip install pymongo

Mongobdb stores BSON.

In [ ]:
from pymongo import MongoClient

In [ ]:
MongoClient('localhost', 27017)

In [ ]:
MongoClient()

In [ ]:
# stkinf - Database
# music - Collection
c = MongoClient().stkinf.music

In [ ]:
c

In [ ]:
jackson = {'name': {'first': 'Michael',
                    'last': 'Jackson',
                    'middle': 'Joseph'},
           'born': datetime(1958, 8, 29),
           'died': datetime(2009, 6, 25),
           'albums': [{'name': 'Thriller',
                       'released': 1982},
                      {'name': 'Bad',
                       'released': 1987}]}

In [ ]:
result = c.insert_one(jackson)

In [ ]:
result

In [ ]:
result.acknowledged

In [ ]:
result.inserted_id

In [ ]:
c.find_one(result.inserted_id)

In [ ]:
c.find_one() # any object

In [ ]:
list(c.find())

In [ ]:
for i in c.find():
    print i

In [ ]:
for i in c.find().limit(10):
    print i

In [ ]:
c.find_one({'born': datetime(1958, 8, 29)})

In [ ]:
c.find_one({'name.middle': 'Joseph'})

In [ ]:
c.find_one({'albums.released': 1982})

In [ ]:
c.find_one({'albums.released': {'$gt': 1980}})

In [ ]:
# AND
c.find_one({'name.first': 'Michael',
             'name.last': 'Bowie'})

In [ ]:
# AND
c.find_one({'$or': [{'name.first': 'Michael'},
                     {'name.last': 'Bowie'}]})

In [ ]:
bowie = {'name': {'first': 'David',
                  'last': 'Bowie',
                  'middle': 'Robert'},
         'born': datetime(1049, 1, 8)}

In [ ]:
c.insert_one(bowie)

In [ ]:
list(c.find({}, {'name.first': 1,
                 'born': 1}))

In [ ]:
list(c.find({}, {'name.first': 1,
                 'born': 1,
                 '_id': 0}))

In [ ]:
list(c.find({}, {'_id': 0}))

In [ ]:
c.find_one({'born': {'$lt': datetime(2000, 1, 1)}, # AND
            'born': {'$gt': datetime(1900, 1, 1)}})

# Modifying things

In [ ]:
result = c.update_one({'name.last': 'Bowie'},
                      {'$set': {'albums': []}})

In [ ]:
result.acknowledged

In [ ]:
c.find_one({'name.last': 'Bowie'})

In [ ]:
c.update_one({'name.last': 'Bowie'},
             {'$push': {'albums': {'name': "Let's Dance",
                                   'released': 1983}}})

In [ ]:
c.find_one({'name.last': 'Bowie'})

In [ ]:
c.update_one({'name.last': 'Bowie'},
             {'$inc': {'albums.0.released': 1}})

In [ ]:
c.find_one({'name.last': 'Bowie'})

In [ ]:
r = c.update_one({'name.last': 'Bowie'},
                 {'$inc': {'albums.0.released': -1}})

## Aggregation

We'll talk more about this next session.

In [ ]:
agg = c.aggregate([{'$group': {'_id': '$born',
                               'people_count': {'$sum': 1}}}])

In [ ]:
list(agg)